# HW:請自選網站，透過爬蟲城市，將91app資料，新增一項以上的外部欄位
寫一支591租屋網的爬蟲，以比較不同區域的租屋數量與91app資料的關係
* 租屋可能以年輕族群居多，租屋多的地方或許購買力較高!?
* 租屋族可能比較容易沒有管理室，或許送貨到便利商店的比例較高!?

In [1]:
import requests
import json
import pandas as pd

# 591租屋網爬蟲

## 觀察的結果：
* 591資料是json格式，存放的位置在"https://rent.591.com.tw/home/search/rsListis_new_list=1&type=1&kind=0&searchtype=1&firstRow=1"
* 若要更換查詢的地區(如：台北市、新北市、桃園市)並非由url控制，而是須更換cookie之中的"urlJumpIP"
* 591上的資料每一頁會有30筆租屋資訊，可以藉由遞增url中的"firstRow"來獲取資料
* 若要獲得每一地區的總共的租屋筆數，則可從網頁原始碼中的records獲得

## 實作的邏輯：
* 最外層先用地區的迴圈(台北市→基隆市→新北市.....)
* 內層在用某地區租屋資料筆數的迴圈
* 最後再寫入CSV存檔，就可以獲得當天591租屋網上全台灣的租屋資訊

In [ ]:
headers = {"User-Agent": "Mozilla/52.0.2 Chrome/57.0.2987.133 Safari/11602.4.8.0.1"}
original_data =[]
count=0
for city in range(1,26):
    if city not in (9,16,18,20): #移除重複的"台北市"資料
        cookie = {'urlJumpIp':str(city)} #地區的選擇是由cookie控制
        records_req= requests.get("https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1", headers=headers,cookies=cookie)
        records = int(json.loads(records_req.text)['records'].replace(",",'')) #移除千分位逗號
        for n in range(0,records,30):
            url = "https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&firstRow="+str(n)
            res = requests.get(url, headers=headers,cookies=cookie)
            original_data += json.loads(res.text)['data']['data']
    data591 = pd.DataFrame(original_data)
    data591.to_csv("591_data.csv_1",encoding = "utf-8-sig")

# 591租屋資料
* 目前只爬了一天的資料可能還不夠具有代表性
* 爬下來的資訊很多，大部分目前還用不太到，或許未來能有其他的利用

In [3]:
#資料讀取
data591 = pd.read_csv("591_data.csv",encoding = "utf-8-sig")

In [4]:
#資料型態
data591.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49022 entries, 0 to 49021
Data columns (total 89 columns):
Unnamed: 0                49022 non-null int64
addInfo                   6952 non-null object
addition2                 49022 non-null int64
addition3                 49022 non-null int64
addition4                 49022 non-null int64
addr_number_name          13723 non-null object
address                   49022 non-null object
address_img               49022 non-null object
address_img_title         49022 non-null object
alley_name                16443 non-null object
allfloor                  49022 non-null int64
area                      49022 non-null float64
balcony_area              0 non-null float64
browsenum                 49022 non-null int64
browsenum_all             49022 non-null int64
browsenum_name            49022 non-null object
cartplace                 0 non-null float64
cases_id                  35478 non-null float64
cases_name                22255 non-nul

In [5]:
#資料概覽
data591.head(3).T

,0,1,2
Unnamed: 0,0,1,2
addInfo,NaN,NaN,NaN
addition2,0,0,0
addition3,0,0,0
addition4,1,1,1
addr_number_name,NaN,14號,306號
address,紹興北街31巷國際交..,德行東路74巷自租雅房房生..,忠孝東路四段僑安地下停車..
address_img,國際交流ShareHouse-善導寺1,自租雅房房生活機能佳環境安靜交通便利,僑安地下停車場近仁愛路、忠孝東路
address_img_title,國際交流ShareHouse-善導寺1,自租雅房房生活機能佳環境安靜交通便利,僑安地下停車場近仁愛路、忠孝東路
alley_name,31巷,74巷,NaN


# 91APP資料

In [6]:
#資料讀取
Orders = pd.read_csv('datasets/Orders.csv', low_memory=False)

In [7]:
#資料清洗(去除City之中有無法識別的城市資料及地區為空值的資料)
Orders.City.unique()

array(['嘉義縣', '桃園市', '台南市', '新北市', '新竹市', '台北市', '苗栗縣', '台中市', '高雄市',
       '彰化縣', '屏東縣', '新竹縣', '宜蘭縣', '花蓮縣', '雲林縣', '台東縣', '基隆市', '南投縣',
       '嘉義市', '台東市', '澎湖縣', '屏東市', 'New', '<fo', 'Kao', 'Kot', 'Tao',
       'Hy ', nan, '25 ', 'Keb', '1st', 'Tai', 'Ins', '桃園縣', 'Đại', 'Cha',
       'Xin', 'Quậ', 'Nan', 'No.', 'Hsi', 'Daf', 'Aya', 'Jia'], dtype=object)

In [8]:
drop = ['New','<fo', 'Kao', 'Kot', 'Tao','Hy ', '25 ', 'Keb', '1st', 'Tai', 'Ins', 'New', '<fo', 'Kao', 'Kot', 'Tao','Hy ','25 ', 'Keb', '1st', 'Tai', 'Ins',]
Orders = Orders[(-Orders["City"].isin(drop)) & (Orders.City.notnull()) & (Orders.District.notnull())]

# 資料合併

In [10]:
dataByCity = {}
dataByCity["Rent"] = {}
dataByCity["91app"] = {}
dataByCity["91app_store"] = {}
for city in data591.regionname.unique():
    dict591 = dict(data591[data591.regionname == city].section_name.value_counts())
    dict91app =  dict(Orders[Orders.City == city].District.value_counts())
    dict91appStore = dict(Orders[(Orders.City == city)&(Orders.DeliverTypeDef !='Home')].District.value_counts())
    for d1 in dict591:
        dataByCity["Rent"][(city,d1)] = dict591[d1]
    for d2 in dict91app:
        dataByCity["91app"][(city,d2)] = dict91app[d2]
    for d3 in dict91appStore:
        dataByCity["91app_store"][(city,d3)] = dict91appStore[d3]

In [11]:
df = pd.DataFrame(dataByCity)
df

Rent    91app  91app_store
南投縣 中寮鄉     1.0    375.0        311.0
    仁愛鄉     1.0   1394.0       1193.0
    信義鄉     NaN    766.0        725.0
    南投市    61.0  12781.0      11736.0
    名間鄉     4.0   2099.0       1865.0
    國姓鄉     2.0    783.0        724.0
    埔里鎮    35.0  12960.0      12191.0
    水里鄉     NaN   1397.0       1257.0
    竹山鎮     7.0   5240.0       4739.0
    草屯鎮    62.0  12508.0      11337.0
    集集鎮     1.0    954.0        862.0
    魚池鄉     5.0   2014.0       1888.0
    鹿谷鄉     3.0   1042.0        918.0
台中市 中區    199.0   6135.0       5593.0
    北區   1773.0  56589.0      50226.0
    北屯區  1215.0  58133.0      50729.0
    南區    680.0  39159.0      34212.0
    南屯區   639.0  41771.0      35399.0
    后里區    33.0   7212.0       6603.0
    和平區     NaN    434.0        298.0
    外埔區     5.0   2617.0       2327.0
    大安區     NaN   1012.0        868.0
    大甲區    46.0  10664.0       9844.0
    大肚區    11.0   8017.0       7440.0
    大里區   227.0  39065.0      35337.0
    大雅區   103.0  18775.0      16948.0
    太平區   241.0  36143.0      32887.0
    新社區     1.0   2353.0       2213.0
    東勢區     9.0   4460.0       4137.0
    東區    278.0  14456.0      13172.0
...         ...      ...          ...
高雄市 大寮區    83.0  17107.0      15778.0
    大樹區     6.0   5807.0       5385.0
    大社區    86.0   7165.0       6561.0
    小港區   140.0  22643.0      20866.0
    岡山區    60.0  14008.0      12867.0
    左營區   421.0  35705.0      30941.0
    彌陀區     NaN   1688.0       1567.0
    新興區   260.0  10123.0       8394.0
    旗山區    12.0   3569.0       3433.0
    旗津區     5.0   2372.0       2191.0
    杉林區     1.0    270.0        245.0
    林園區    21.0   7050.0       6466.0
    梓官區    10.0   4072.0       3876.0
    楠梓區   448.0  39267.0      35660.0
    橋頭區    21.0   4185.0       3649.0
    永安區     2.0    619.0        567.0
    湖內區    44.0   4609.0       4362.0
    燕巢區    32.0   8128.0       7686.0
    田寮區     NaN     28.0          NaN
    甲仙區     NaN    222.0        219.0
    美濃區     4.0   2134.0       2052.0
    苓雅區   469.0  28014.0      24231.0
    茂林區     NaN      7.0          NaN
    茄萣區     5.0   2992.0       2842.0
    路竹區    41.0   7451.0       6564.0
    阿蓮區     1.0   2712.0       2424.0
    鳥松區    47.0   6820.0       5935.0
    鳳山區   313.0  49982.0      44640.0
    鹽埕區    85.0   4117.0       3801.0
    鼓山區   304.0  24124.0      20717.0

[357 rows x 3 columns]